In [2]:
import numpy as np
import pandas as pd
import requests
import json

In [3]:
gr = pd.read_csv('cleaned_goodreads.csv', index_col=0)
gr

,book_id,overall,reviewTime,asin,reviewText
0,22551730,4,"Dec 14, 2016",0307408868,Another hard to put down nonfiction book from ...
1,18176747,5,"Dec 21, 2016",0062273205,I haven't read many (any?) books that are writ...
2,137554,0,"Mar 20, 2014",006073731X,Sacca and Nate recommend
3,40955,5,"Dec 21, 2016",0071424911,A truly inspirational book by a truly inspirat...
4,9850443,3,"Aug 05, 2012",0062041266,"A fun, dark, slightly comical western about tw..."
...,...,...,...,...,...
960443,4405141,3,"Aug 19, 2014",0061698954,While i liked it and appreciated all the infor...
960444,4405141,5,"Apr 15, 2013",0061698954,If you know anyone suffering from an eating di...
960445,4405141,5,"Jul 28, 2015",0061698954,Fabulous insight to what people struggling wit...
960446,4405141,5,"Mar 30, 2009",0061698954,This is an excellent resource -best book I hav...


In [4]:
books = np.unique(gr.asin)
print(len(books))

37271


In [5]:
# Goodreads API Developer key
key = 'mlUZUnPildWfc5tlXU03w'
secret = 'vHaBborkweWsa1J2hB5IN6E4jDM7WlL3Xm0D2bcWAI'

In [57]:
df = pd.DataFrame(columns = ['asin', 'ratings_count', 'reviews_count', 'text_reviews_count', 
                             'total_ratings_count', 'total_reviews_count', 'total_text_reviews_count', 'average_rating'])

In [59]:
count = 0 # Used to get 300 ISBNS at once (Goodreads limit)
while count < len(books):
    # API request
    res = requests.get('https://www.goodreads.com/book/review_counts.json', params={'key': key, 'isbns[]': books[count:(count+300)]})
    # Convert result to JSON
    data = res.json()
    # Parse JSON into dataframe
    for i,val in enumerate(data['books']):
        df = df.append({'asin':books[count+i], 'ratings_count':val['ratings_count'], 'reviews_count':val['reviews_count'], 
                        'text_reviews_count':val['text_reviews_count'], 'total_ratings_count':val['work_ratings_count'],
                        'total_reviews_count':val['work_reviews_count'], 
                        'total_text_reviews_count':val['work_text_reviews_count'], 'average_rating':val['average_rating']},
                       ignore_index=True)
                
    count += 300

In [60]:
df

,asin,ratings_count,reviews_count,text_reviews_count,total_ratings_count,total_reviews_count,total_text_reviews_count,average_rating
0,000100039X,186297,163625,5535,220088,196528,8847,4.23
1,0001053655,16,33,6,676,1552,85,4.08
2,0001061240,10,22,2,221,603,36,4.62
3,000161102X,33,74,4,2929,5786,75,3.86
4,0001711296,604,1319,48,738,1564,65,4.29
...,...,...,...,...,...,...,...,...
37262,0312953240,75,178,8,87,201,8,3.80
37263,0312955138,27,75,3,29,81,4,3.38
37264,0312955154,52,94,5,56,101,5,3.36
37265,0312956878,57,189,3,59,193,4,3.78


In [61]:
df.to_csv('goodreads_metadata.csv')